# Revising manuscript. 
## Data preparing according to reviewer's comments.

In [1]:
import pandas as pd
import json
from pprint import pprint

In [2]:
cre = pd.read_csv('ephys_data_cre_EI_180116.csv') # use this data for using specimen id
# cre = pd.read_csv('ephys_data_cre.csv')
# as review 2 minor commemts # 11 . technical values removed.
cre = cre.drop(['reporter_status', 'height', 'input_resistance_mohm', 'vm_for_sag', 
                'slow_trough_t_long_square', 'slow_trough_v_long_square',
               'slow_trough_t_ramp', 'slow_trough_v_ramp'], axis=1)
#cre.info()

adaptation                                816
avg_isi                                   889

In [3]:
# from allensdk.core.cell_types_cache import CellTypesCache
# from allensdk.api.queries.cell_types_api import CellTypesApi
# from allensdk.core.cell_types_cache import ReporterStatus as RS
# ctc = CellTypesCache(manifest_file='cell_types/manifest.json')
# cells = ctc.get_cells()
with open('cell_types/cells.json') as df:
    cells = json.load(df)
print("All cells: %d" % len(cells)) # load data, for including layer metadata.

All cells: 1337


In [4]:
cell_index = { c['id']: c for c in cells }

In [6]:
tau = [cell_index[cid]['ephys_features'][0]['tau'] for cid in cre['specimen_id']]
cre['tau'] = pd.Series(tau, index=cre.index)

In [7]:

# cre.head()
# cell_index[489923501]['structure']['safe_name']

In [8]:
layer_full = [cell_index[cid]['structure']['safe_name'] for cid in cre['specimen_id']]
cre['layer_full'] = pd.Series(layer_full, index=cre.index)
# cre.head()

In [9]:
layer_acronym = [cell_index[cid]['structure']['acronym'] for cid in cre['specimen_id']]
cre['acronym'] = pd.Series(layer_acronym, index=cre.index)
# cre.head()

In [10]:
cre_dropna = cre.dropna(axis=0, how='any') # before fill the adatation index, avg_isi

In [12]:
len(cre_dropna)

792

In [13]:
a = cre['transgenic_line'].value_counts()
b= cre_dropna['transgenic_line'].value_counts()
a - b

Chat-IRES-Cre-neo    11
Chrna2-Cre_OE25       2
Ctgf-T2A-dgCre        0
Cux2-CreERT2         13
Gad2-IRES-Cre         4
Htr3a-Cre_NO152      18
Ndnf-IRES2-dgCre      1
Nkx2-1-CreERT2        1
Nr5a1-Cre             9
Ntsr1-Cre_GN220      15
Pvalb-IRES-Cre       38
Rbp4-Cre_KL100       13
Rorb-IRES2-Cre       14
Scnn1a-Tg2-Cre        4
Scnn1a-Tg3-Cre        7
Slc17a6-IRES-Cre      1
Sst-IRES-Cre         25
Vip-IRES-Cre          1
Name: transgenic_line, dtype: int64

In [14]:
(a-b)/a # before fill na value in adaptation and avg_isi

Chat-IRES-Cre-neo    0.550000
Chrna2-Cre_OE25      0.064516
Ctgf-T2A-dgCre       0.000000
Cux2-CreERT2         0.178082
Gad2-IRES-Cre        0.363636
Htr3a-Cre_NO152      0.202247
Ndnf-IRES2-dgCre     0.066667
Nkx2-1-CreERT2       0.043478
Nr5a1-Cre            0.130435
Ntsr1-Cre_GN220      0.312500
Pvalb-IRES-Cre       0.267606
Rbp4-Cre_KL100       0.180556
Rorb-IRES2-Cre       0.116667
Scnn1a-Tg2-Cre       0.111111
Scnn1a-Tg3-Cre       0.116667
Slc17a6-IRES-Cre     0.333333
Sst-IRES-Cre         0.225225
Vip-IRES-Cre         0.050000
Name: transgenic_line, dtype: float64

In [22]:
(cre['slow_trough_t_short_square'] == cre['trough_t_short_square']).value_counts() # almost same properties

True     954
False     15
dtype: int64

In [23]:
(cre['slow_trough_v_short_square'] == cre['trough_v_short_square']).value_counts() # almost same properties

True     878
False     91
dtype: int64

In [24]:
cre = cre.drop(['slow_trough_v_short_square', 'slow_trough_t_short_square'], axis=1)

In [25]:
cre_fill = cre # make new dataset for fill NA values

In [26]:
cre_fill['adaptation'] = cre_fill['adaptation'].fillna(1.0) # if cells to adapt to stimuli, and can't make next spike. 

In [27]:
cre_fill['avg_isi']=cre_fill['avg_isi'].fillna(1000.0)  # long square stimul duration is 1 sec

In [28]:
cre_fill_dropna = cre_fill.dropna(axis=0, how='any') # after fill the adatation index, avg_isi

In [29]:
a = cre_fill['transgenic_line'].value_counts()
b= cre_fill_dropna['transgenic_line'].value_counts()
a - b

Chat-IRES-Cre-neo     1
Chrna2-Cre_OE25       0
Ctgf-T2A-dgCre        0
Cux2-CreERT2          0
Gad2-IRES-Cre         1
Htr3a-Cre_NO152       3
Ndnf-IRES2-dgCre      0
Nkx2-1-CreERT2        1
Nr5a1-Cre             1
Ntsr1-Cre_GN220       0
Pvalb-IRES-Cre       23
Rbp4-Cre_KL100        0
Rorb-IRES2-Cre        0
Scnn1a-Tg2-Cre        0
Scnn1a-Tg3-Cre        0
Slc17a6-IRES-Cre      0
Sst-IRES-Cre          5
Vip-IRES-Cre          0
Name: transgenic_line, dtype: int64

In [30]:
(a-b)/a # before fill na value in adaptation and avg_isi

Chat-IRES-Cre-neo    0.050000
Chrna2-Cre_OE25      0.000000
Ctgf-T2A-dgCre       0.000000
Cux2-CreERT2         0.000000
Gad2-IRES-Cre        0.090909
Htr3a-Cre_NO152      0.033708
Ndnf-IRES2-dgCre     0.000000
Nkx2-1-CreERT2       0.043478
Nr5a1-Cre            0.014493
Ntsr1-Cre_GN220      0.000000
Pvalb-IRES-Cre       0.161972
Rbp4-Cre_KL100       0.000000
Rorb-IRES2-Cre       0.000000
Scnn1a-Tg2-Cre       0.000000
Scnn1a-Tg3-Cre       0.000000
Slc17a6-IRES-Cre     0.000000
Sst-IRES-Cre         0.045045
Vip-IRES-Cre         0.000000
Name: transgenic_line, dtype: float64

In [32]:
len(cre_fill_dropna) # before drop slow_trough, so recheck the value. after drop slow_trough col.
#remain 934 cells.

934

In [33]:
cre = cre_fill # replace the filled dataset

In [34]:
cre.loc[cre['transgenic_line'] == 'Ndnf-IRES2-dgCre', ['binary_neuron']] = 'Inhibitory'
cre.loc[cre['transgenic_line'] == 'Ndnf-IRES2-dgCre']

,specimen_id,adaptation,avg_isi,f_i_curve_slope,fast_trough_t_long_square,fast_trough_t_ramp,fast_trough_t_short_square,fast_trough_v_long_square,fast_trough_v_ramp,fast_trough_v_short_square,...,trough_v_short_square,upstroke_downstroke_ratio_long_square,upstroke_downstroke_ratio_ramp,upstroke_downstroke_ratio_short_square,vrest,transgenic_line,binary_neuron,tau,layer_full,acronym
70,526943222,-0.016836,51.317500,0.253219,1.53066,4.356407,1.025040,-53.656254,-53.114587,-56.575002,...,-68.137506,3.277697,3.293132,2.619381,-67.046272,Ndnf-IRES2-dgCre,Inhibitory,19.153995,Primary visual area layer 1,VISp1
99,527089085,0.008434,36.099259,0.480000,1.04764,7.642927,1.027312,-47.437504,-46.395836,-56.981253,...,-73.237500,2.780688,3.089430,2.601046,-72.885735,Ndnf-IRES2-dgCre,Inhibitory,9.042324,Primary visual area layer 1,VISp1
258,527579814,0.005303,52.404706,0.328125,1.83288,8.159487,1.027337,-49.937500,-51.989586,-60.036462,...,-76.656253,3.979260,4.137271,2.575332,-76.635284,Ndnf-IRES2-dgCre,Inhibitory,4.763883,Primary visual area layer 1,VISp1
264,530235049,0.002679,27.175000,0.335919,1.11192,7.368600,1.024668,-53.500004,-52.239586,-55.675002,...,-70.387503,2.502756,2.628995,2.568368,-69.332764,Ndnf-IRES2-dgCre,Inhibitory,8.629247,Primary visual area layer 2/3,VISp2/3
356,564346637,1.000000,20.160000,0.665800,1.06240,6.785400,1.024127,-60.875004,-52.406250,-61.963545,...,-63.812503,1.480181,1.053327,1.613715,-62.698147,Ndnf-IRES2-dgCre,Inhibitory,25.978014,Primary visual area layer 5,VISp5
456,526531616,0.006942,28.112353,0.303640,1.53710,6.348080,1.026160,-53.437500,-50.052087,-55.975002,...,-72.425003,2.980973,2.859881,2.546601,-72.557991,Ndnf-IRES2-dgCre,Inhibitory,9.050988,Primary visual area layer 1,VISp1
460,579611597,-0.025917,79.514545,0.257085,1.85968,9.415487,1.027660,-52.406250,-49.739585,-58.562504,...,-76.406258,3.787364,3.863806,2.875604,-75.740387,Ndnf-IRES2-dgCre,Inhibitory,8.572176,Primary visual area layer 2/3,VISp2/3
525,527869035,-0.000414,31.932143,0.391826,1.96766,6.447647,1.025717,-50.656250,-51.541669,-55.088544,...,-65.880213,3.023153,3.061118,2.408996,-65.245323,Ndnf-IRES2-dgCre,Inhibitory,8.652592,Primary visual area layer 1,VISp1
574,526939383,0.002607,31.416129,0.340848,1.09790,6.254480,1.025248,-48.968750,-49.156253,-51.343752,...,-72.431255,2.717109,2.526602,2.358213,-71.806557,Ndnf-IRES2-dgCre,Inhibitory,10.630557,Primary visual area layer 1,VISp1
704,527464013,-0.007651,37.511200,0.312106,1.47392,8.514600,1.028415,-52.656254,-49.458336,-60.109377,...,-75.367189,3.007957,3.439152,2.371974,-75.434464,Ndnf-IRES2-dgCre,Inhibitory,5.897970,Primary visual area layer 1,VISp1


In [35]:
cre.loc[cre['binary_neuron'] == 'Excitatory', 'transgenic_line'].value_counts()

Rorb-IRES2-Cre      120
Cux2-CreERT2         73
Rbp4-Cre_KL100       72
Nr5a1-Cre            69
Scnn1a-Tg3-Cre       60
Ntsr1-Cre_GN220      48
Scnn1a-Tg2-Cre       36
Ctgf-T2A-dgCre       26
Slc17a6-IRES-Cre      3
Name: transgenic_line, dtype: int64

In [36]:
cre[cre['transgenic_line'] == 'Ctgf-T2A-dgCre']['layer_full'].value_counts()

Posterolateral visual area layer 6b        8
Primary visual area layer 6b               7
Lateral visual area layer 6b               3
Primary visual area layer 6a               3
Primary visual area layer 5                2
Laterointermediate area layer 6b           1
Anterior area layer 6b                     1
Retrosplenial area dorsal part layer 6a    1
Name: layer_full, dtype: int64

In [37]:
cre[cre['transgenic_line'] == 'Cux2-CreERT2']['layer_full'].value_counts()

Primary visual area layer 4          36
Primary visual area layer 2/3        21
Primary visual area layer 5          11
Primary visual area layer 1           1
posteromedial visual area layer 4     1
posteromedial visual area layer 5     1
Postrhinal area layer 5               1
Lateral visual area layer 4           1
Name: layer_full, dtype: int64

In [38]:
cre[cre['transgenic_line'] == 'Nr5a1-Cre']['layer_full'].value_counts()

Primary visual area layer 4            44
Primary visual area layer 2/3          13
posteromedial visual area layer 4       5
Primary visual area layer 5             2
Primary visual area layer 6a            2
Anterolateral visual area layer 4       1
posteromedial visual area layer 2/3     1
Anteromedial visual area layer 2/3      1
Name: layer_full, dtype: int64

In [39]:
cre[cre['transgenic_line'] == 'Ntsr1-Cre_GN220']['layer_full'].value_counts()

Primary visual area layer 6a         42
Primary visual area layer 5           2
posteromedial visual area layer 5     1
Lateral visual area layer 6a          1
Lateral visual area layer 5           1
Primary visual area layer 6b          1
Name: layer_full, dtype: int64

In [40]:
cre[cre['transgenic_line'] == 'Rbp4-Cre_KL100']['layer_full'].value_counts()

Primary visual area layer 5          48
Primary visual area layer 4          18
Primary visual area layer 6a          2
Lateral visual area layer 5           1
posteromedial visual area layer 5     1
Primary visual area layer 2/3         1
Lateral visual area layer 4           1
Name: layer_full, dtype: int64

In [41]:
cre[cre['transgenic_line'] == 'Rorb-IRES2-Cre']['layer_full'].value_counts()

Primary visual area layer 4                            68
Primary visual area layer 5                            27
Primary visual area layer 6a                            6
Primary visual area layer 2/3                           5
Lateral visual area layer 4                             4
Lateral visual area layer 5                             2
Laterointermediate area layer 4                         1
Posterior auditory area layer 5                         1
Posterolateral visual area layer 4                      1
Posterolateral visual area layer 5                      1
posteromedial visual area layer 4                       1
posteromedial visual area layer 5                       1
Postrhinal area layer 5                                 1
Retrosplenial area lateral agranular part layer 2/3     1
Name: layer_full, dtype: int64

In [42]:
cre[cre['transgenic_line'] == 'Scnn1a-Tg2-Cre']['layer_full'].value_counts()

Primary visual area layer 4          26
Primary visual area layer 5           8
posteromedial visual area layer 4     1
posteromedial visual area layer 5     1
Name: layer_full, dtype: int64

In [43]:
cre[cre['transgenic_line'] == 'Scnn1a-Tg3-Cre']['layer_full'].value_counts()

Primary visual area layer 4          38
Primary visual area layer 5          11
Postrhinal area layer 5               3
Primary visual area layer 2/3         3
posteromedial visual area layer 5     2
Laterointermediate area layer 5       1
Lateral visual area layer 4           1
Anteromedial visual area layer 4      1
Name: layer_full, dtype: int64

In [44]:
cre[cre['transgenic_line'] == 'Slc17a6-IRES-Cre']['layer_full'].value_counts()

Primary visual area layer 2/3    2
Primary visual area layer 4      1
Name: layer_full, dtype: int64

In [45]:
# calculate for firing rate.
cre['firing_rate'] = (1/cre['avg_isi'])*1000

In [46]:
cre['peak_t_short_square']= (cre['peak_t_short_square']-1.02)*1000
cre['peak_t_short_square']
# make time unit as 3ms, short square pulse stimuli applied 3ms, so the start of stimuli set to zero.

0      3.300000
1      3.480000
2      3.475000
3      3.508333
4      3.560000
5      3.597000
6      3.541000
7      3.311000
8      3.390000
9      3.570000
10     3.478000
11     3.030625
12     3.508333
13     3.841667
14     3.140000
15     3.462000
16     3.427500
17     3.160000
18     3.318750
19     3.580000
20     3.336000
21     3.432000
22     3.256667
23     3.460000
24     3.657000
25     3.525000
26     3.320000
27     3.673333
28     3.672000
29     3.344000
         ...   
939    3.430000
940    3.392000
941    3.365000
942    3.735000
943    3.420000
944    3.373000
945    3.436250
946    3.405000
947    3.410000
948    3.280000
949    3.515000
950    3.385000
951    3.342500
952    3.215000
953    3.227000
954    3.710000
955    3.448750
956    3.315000
957    3.552000
958    3.524000
959    3.343750
960    3.292000
961    3.447500
962    3.303000
963    3.493750
964    3.413750
965    3.576000
966    3.410000
967    3.445000
968    3.592000
Name: peak_t_short_squar

In [47]:
cre['threshold_t_short_square'] = (cre['threshold_t_short_square']-1.02)*1000
cre['threshold_t_short_square']

0      2.913333
1      3.159000
2      3.001000
3      3.130000
4      3.120000
5      2.979000
6      3.197000
7      3.032000
8      3.010000
9      3.142000
10     2.943000
11     2.731875
12     3.150000
13     3.266667
14     2.917000
15     2.979000
16     3.043750
17     2.746667
18     3.002500
19     3.103000
20     2.946000
21     3.123000
22     2.856667
23     3.104000
24     3.137000
25     3.155000
26     2.970000
27     3.191667
28     3.168000
29     2.985000
         ...   
939    3.055000
940    3.055000
941    2.948333
942    3.289000
943    3.092500
944    3.033000
945    3.061250
946    3.046000
947    3.025000
948    2.970000
949    3.114000
950    3.055000
951    3.040000
952    2.895000
953    2.994000
954    3.190000
955    3.090000
956    3.053000
957    3.120000
958    3.144000
959    2.983750
960    3.008000
961    3.077500
962    3.033000
963    3.192500
964    3.025000
965    3.132000
966    3.095000
967    3.065000
968    3.164000
Name: threshold_t_short_

In [48]:
cre['fast_trough_t_short_square'] = (cre['fast_trough_t_short_square']-1.02)*1000
cre['fast_trough_t_short_square']

0      4.873333
1      4.593000
2      7.302000
3      4.665833
4      5.426667
5      5.615000
6      4.461000
7      4.103000
8      4.870000
9      5.506000
10     5.854000
11     3.790625
12     4.195833
13     5.882500
14     3.625000
15     5.411000
16     4.420000
17     4.846667
18     4.180000
19     5.553000
20     5.011000
21     4.358000
22     4.471667
23     5.428000
24     7.009000
25     4.895000
26     4.625000
27     6.062500
28     5.740000
29     4.485000
         ...   
939    4.815000
940    4.841000
941    5.006667
942    6.266000
943    5.032500
944    4.921000
945    9.810000
946    6.005000
947    5.352500
948    4.520000
949    7.092000
950    4.130000
951    8.495000
952    4.122500
953    3.877000
954    6.587857
955    5.132500
956    3.969000
957    5.192000
958    5.352000
959    5.056250
960    4.348000
961    9.131250
962    3.975000
963    4.292500
964    5.281250
965    5.360000
966    4.603750
967    4.965000
968    5.576000
Name: fast_trough_t_shor

In [49]:
cre['trough_t_short_square'] = (cre['trough_t_short_square']-1.02)*1000
cre['trough_t_short_square']

0      324.913333
1      213.473000
2      671.744000
3      475.571667
4      110.500000
5      453.574000
6      527.561000
7      436.287000
8      489.350000
9      231.151000
10     557.570000
11     343.485625
12     377.621667
13      34.124167
14     673.986000
15     391.247000
16     291.541250
17     811.826667
18     230.333750
19     426.244000
20     592.528000
21     395.886000
22     810.231667
23     585.676000
24     505.379000
25     518.510000
26     835.710000
27     527.079167
28      97.364000
29     720.821000
          ...    
939    669.315000
940    510.804000
941    689.963333
942    338.333000
943    446.345000
944    510.155000
945     27.485000
946    129.956000
947     66.213333
948    585.140000
949    589.005000
950    227.725000
951    179.678750
952      4.257500
953    612.955000
954    618.815000
955    619.258750
956    561.903000
957    584.992000
958    124.844000
959    682.153750
960    458.228000
961    512.405000
962     68.214000
963     26

In [51]:
cre['threshold_t_long_square'] = (cre['threshold_t_long_square']-1.02)*1000
cre['peak_t_long_square'] = (cre['peak_t_long_square']-1.02)*1000
cre['fast_trough_t_long_square'] = (cre['fast_trough_t_long_square']-1.02)*1000
cre['trough_t_long_square'] = (cre['trough_t_long_square']-1.02)*1000
# make time unit as ms, long square pulse stimuli applied for 1 s from 100 ms of sweep
# the start of stimuli set to zero.

In [52]:
cre['threshold_t_ramp'] = cre['threshold_t_ramp'] - 1.02
cre['peak_t_ramp'] = cre['peak_t_ramp'] -1.02
cre['fast_trough_t_ramp'] = cre['fast_trough_t_ramp'] - 1.02
cre['trough_t_ramp'] = cre['trough_t_ramp'] - 1.02
# ramp stimuli applied from 1.0 s of sweep
# the start of stimuli set to zero.

In [54]:
cre['peak_t_long_square'] - cre['threshold_t_long_square']

0      0.420
1      0.390
2      0.570
3      0.415
4      0.540
5      0.665
6      0.385
7      0.320
8      0.420
9      0.525
10     0.630
11     0.330
12     0.380
13     0.680
14     0.260
15     0.510
16     0.400
17     0.500
18     0.325
19     0.615
20     0.470
21     0.335
22     0.470
23     0.440
24     0.725
25     0.420
26     0.390
27     0.495
28     0.660
29     0.400
       ...  
939    0.435
940    0.385
941    0.455
942    0.505
943    0.545
944    0.400
945    0.460
946    0.420
947    0.465
948    0.360
949    0.485
950    0.360
951    0.415
952    0.365
953    0.270
954    0.645
955    0.460
956    0.385
957    0.500
958    0.420
959    0.415
960    0.340
961    0.510
962    0.305
963    0.315
964    0.485
965    0.520
966    0.360
967    0.420
968    0.500
Length: 969, dtype: float64

In [55]:
cre['height_short'] = cre['peak_v_short_square'] -cre['fast_trough_v_short_square']
cre['height_long'] = cre['peak_v_long_square'] - cre['fast_trough_v_long_square']
cre['height_ramp'] = cre['peak_v_ramp'] - cre['fast_trough_v_ramp']

In [56]:
(cre['height_long'] - cre['height_short']).mean

<bound method Series.mean of 0       0.187496
1     -22.099999
2      -2.700003
3      -8.223962
4      -9.635417
5      -6.293752
6      -6.718746
7     -13.662500
8      -4.562498
9      -8.037503
10     -4.874999
11     -3.972647
12     -0.749999
13    -11.859377
14    -13.056254
15     -4.850003
16     -2.234374
17     -2.489583
18     -3.273437
19     -7.656248
20    -10.331248
21     -0.293748
22     -1.500002
23     -8.206251
24    -10.037502
25     -3.007812
26     -2.718750
27     -6.911457
28     -9.443750
29     -1.706247
         ...    
939    -6.218749
940    -6.118752
941     6.052085
942    -7.125002
943   -27.765625
944    -3.981248
945    -4.148438
946    -5.131252
947    -8.656249
948    -2.007816
949    -9.474995
950    -4.367189
951   -11.320311
952   -13.906254
953    -4.537498
954    -7.058038
955    -9.882810
956   -15.675001
957   -10.212505
958    -3.600000
959    -0.414067
960   -11.150001
961   -13.718753
962    -9.350000
963    -7.437502
964   -10.773438
96

In [57]:
(cre['height_long'] - cre['height_ramp']).mean

<bound method Series.mean of 0      -0.833333
1     -20.000001
2      -7.718753
3       3.843746
4      -1.875001
5     -10.916670
6       2.427087
7      -0.020831
8      -3.458332
9     -16.208337
10     -1.562497
11           NaN
12      2.197920
13     -9.645837
14     14.249996
15     -3.635420
16     -4.249997
17     -2.062499
18     -0.552080
19      3.520835
20      7.552085
21      2.875002
22     -0.812500
23     -4.624996
24     -5.104167
25     -0.937500
26     -0.187496
27           NaN
28     -3.979164
29      1.250004
         ...    
939    -1.197917
940     3.781252
941     3.552086
942    -4.250003
943   -15.895835
944    -6.458330
945   -10.729166
946    -2.364583
947    -3.291666
948     0.291660
949    -4.218744
950     4.999998
951    -7.124999
952   -15.687505
953     7.833336
954   -13.145835
955    -0.604164
956     8.656252
957    -4.197919
958    -2.447918
959     0.156244
960     4.250002
961   -10.531254
962    24.125000
963    13.953124
964   -11.302085
96

In [58]:
(cre['height_ramp'] - cre['height_short']).mean

<bound method Series.mean of 0       1.020830
1      -2.099998
2       5.018749
3     -12.067707
4      -7.760415
5       4.622918
6      -9.145833
7     -13.641668
8      -1.104166
9       8.170833
10     -3.312502
11           NaN
12     -2.947919
13     -2.213540
14    -27.306250
15     -1.214583
16      2.015624
17     -0.427085
18     -2.721357
19    -11.177083
20    -17.883333
21     -3.168750
22     -0.687502
23     -3.581255
24     -4.933336
25     -2.070312
26     -2.531254
27           NaN
28     -5.464586
29     -2.956251
         ...    
939    -5.020832
940    -9.900003
941     2.499999
942    -2.874999
943   -11.869790
944     2.477082
945     6.580729
946    -2.766668
947    -5.364583
948    -2.299477
949    -5.256251
950    -9.367187
951    -4.195312
952     1.781251
953   -12.370834
954     6.087796
955    -9.278646
956   -24.331252
957    -6.014586
958    -1.152082
959    -0.570312
960   -15.400003
961    -3.187499
962   -33.475000
963   -21.390626
964     0.528647
96

In [59]:
cre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 969 entries, 0 to 968
Data columns (total 47 columns):
specimen_id                               969 non-null int64
adaptation                                969 non-null float64
avg_isi                                   969 non-null float64
f_i_curve_slope                           969 non-null float64
fast_trough_t_long_square                 969 non-null float64
fast_trough_t_ramp                        934 non-null float64
fast_trough_t_short_square                969 non-null float64
fast_trough_v_long_square                 969 non-null float64
fast_trough_v_ramp                        934 non-null float64
fast_trough_v_short_square                969 non-null float64
latency                                   969 non-null float64
peak_t_long_square                        969 non-null float64
peak_t_ramp                               934 non-null float64
peak_t_short_square                       969 non-null float64
peak_v_long_squ

In [60]:
cre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 969 entries, 0 to 968
Data columns (total 47 columns):
specimen_id                               969 non-null int64
adaptation                                969 non-null float64
avg_isi                                   969 non-null float64
f_i_curve_slope                           969 non-null float64
fast_trough_t_long_square                 969 non-null float64
fast_trough_t_ramp                        934 non-null float64
fast_trough_t_short_square                969 non-null float64
fast_trough_v_long_square                 969 non-null float64
fast_trough_v_ramp                        934 non-null float64
fast_trough_v_short_square                969 non-null float64
latency                                   969 non-null float64
peak_t_long_square                        969 non-null float64
peak_t_ramp                               934 non-null float64
peak_t_short_square                       969 non-null float64
peak_v_long_squ

In [61]:
cre['transgenic_line'].value_counts()

Pvalb-IRES-Cre       142
Rorb-IRES2-Cre       120
Sst-IRES-Cre         111
Htr3a-Cre_NO152       89
Cux2-CreERT2          73
Rbp4-Cre_KL100        72
Nr5a1-Cre             69
Scnn1a-Tg3-Cre        60
Ntsr1-Cre_GN220       48
Scnn1a-Tg2-Cre        36
Chrna2-Cre_OE25       31
Ctgf-T2A-dgCre        26
Nkx2-1-CreERT2        23
Vip-IRES-Cre          20
Chat-IRES-Cre-neo     20
Ndnf-IRES2-dgCre      15
Gad2-IRES-Cre         11
Slc17a6-IRES-Cre       3
Name: transgenic_line, dtype: int64

In [62]:
cre[cre['binary_neuron'] == 'Excitatory']['layer_full'].value_counts()

Primary visual area layer 4                            231
Primary visual area layer 5                            111
Primary visual area layer 6a                            55
Primary visual area layer 2/3                           45
Posterolateral visual area layer 6b                      8
posteromedial visual area layer 4                        8
Primary visual area layer 6b                             8
posteromedial visual area layer 5                        7
Lateral visual area layer 4                              7
Postrhinal area layer 5                                  5
Lateral visual area layer 5                              4
Lateral visual area layer 6b                             3
posteromedial visual area layer 2/3                      1
Posterolateral visual area layer 4                       1
Laterointermediate area layer 6b                         1
Anteromedial visual area layer 2/3                       1
Anteromedial visual area layer 4                        

In [63]:
cre[cre['binary_neuron'] == 'Excitatory']['acronym'].value_counts()

VISp4        231
VISp5        111
VISp6a        55
VISp2/3       45
VISpl6b        8
VISp6b         8
VISpm4         8
VISpm5         7
VISl4          7
VISpor5        5
VISl5          4
VISl6b         3
RSPd6a         1
VISam4         1
RSPagl2/3      1
AUDpo5         1
VISp1          1
VISam2/3       1
VISli6b        1
VISli4         1
VISli5         1
VISpl5         1
VISal4         1
VISa6b         1
VISpm2/3       1
VISpl4         1
VISl6a         1
Name: acronym, dtype: int64

In [65]:
cre.head()

,specimen_id,adaptation,avg_isi,f_i_curve_slope,fast_trough_t_long_square,fast_trough_t_ramp,fast_trough_t_short_square,fast_trough_v_long_square,fast_trough_v_ramp,fast_trough_v_short_square,...,vrest,transgenic_line,binary_neuron,tau,layer_full,acronym,firing_rate,height_short,height_long,height_ramp
0,569653118,0.064384,76.283333,0.195247,82.66,2.985247,4.873333,-48.718750,-49.875001,-51.500004,...,-66.085068,Chrna2-Cre_OE25,Inhibitory,12.149962,Lateral visual area layer 5,VISl5,13.109023,105.031258,105.218754,106.052087
1,485466109,0.048709,118.397143,0.307589,49.77,6.183945,4.593000,-45.968754,-49.093754,-53.800002,...,-65.503044,Sst-IRES-Cre,Inhibitory,13.757958,Primary visual area layer 2/3,VISp2/3,8.446150,81.287503,59.187504,79.187505
2,483057325,0.042215,81.394545,0.162729,69.75,1.316427,7.302000,-48.906250,-51.083335,-55.800002,...,-72.174675,Cux2-CreERT2,Excitatory,21.945011,Primary visual area layer 4,VISp4,12.285836,84.887503,82.187500,89.906253
3,488440436,0.002560,18.282170,0.656473,30.44,6.500155,4.665833,-59.500000,-58.125004,-61.609378,...,-73.008224,Pvalb-IRES-Cre,Inhibitory,8.355113,Primary visual area layer 5,VISp5,54.698103,88.661462,80.437500,76.593754
4,501566512,0.542358,307.100000,0.155347,65.16,7.516707,5.426667,-47.000000,-46.927086,-55.093753,...,-80.050621,Ntsr1-Cre_GN220,Excitatory,14.181895,Primary visual area layer 6a,VISp6a,3.256268,97.416670,87.781254,89.656255


In [66]:
VISp2 = cre['acronym'] == 'VISp2/3'
VISp4 = cre['acronym'] == 'VISp4'
VISp5 = cre['acronym'] == 'VISp5'
VISp6a = cre['acronym'] == 'VISp6a'
VISp6b = cre['acronym'] == 'VISp6b'
Ctgf = cre['transgenic_line'] == 'Ctgf-T2A-dgCre'
Cux2 = cre['transgenic_line'] == 'Cux2-CreERT2'
Nr5a1= cre['transgenic_line'] == 'Nr5a1-Cre'
Ntsr1 = cre['transgenic_line'] == 'Ntsr1-Cre_GN220'
Rorb = cre['transgenic_line'] == 'Rorb-IRES2-Cre'
ScnTg2 = cre['transgenic_line'] == 'Scnn1a-Tg2-Cre'
ScnTg3 = cre['transgenic_line'] == 'Scnn1a-Tg3-Cre'
Slc = cre['transgenic_line'] == 'Slc17a6-IRES-Cre'
Rbp4 = cre['transgenic_line'] == 'Rbp4-Cre_KL100'

In [67]:
Rbp4.mean()

0.074303405572755415

In [68]:
L2 = cre[VISp2 & (Cux2 | Slc | Rbp4)]
L2['L_tg'] = "L2tg"

/home/hyunsu/anaconda3/envs/allen/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [69]:
len(L2)

24

In [70]:
L4 = cre[VISp4 & (Nr5a1 | Rbp4 | Rorb | ScnTg2 | ScnTg3)]
L4['L_tg'] = "L4tg"

/home/hyunsu/anaconda3/envs/allen/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [71]:
len(L4)

194

In [72]:
L5 = cre[VISp5 & (Cux2 | Rbp4 | Rorb | ScnTg2 | ScnTg3)]
L5['L_tg'] = "L5tg"

/home/hyunsu/anaconda3/envs/allen/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [73]:
len(L5)

105

In [74]:
L6a = cre[VISp6a & (Ntsr1 | Rbp4 | Rorb | ScnTg3)]
L6a['L_tg'] = "L6atg"

/home/hyunsu/anaconda3/envs/allen/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [75]:
len(L6a)

50

In [76]:
L6b = cre[VISp6b & Ctgf]
L6b['L_tg'] = "L6btg"

/home/hyunsu/anaconda3/envs/allen/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [77]:
len(L6b)

7

In [78]:
layer_data = pd.concat([L2, L4, L5, L6a, L6b])

In [79]:
len(layer_data)

380

In [80]:
cre.to_csv('cre_revised.csv', index = False)

In [81]:
layer_data.to_csv('layer_revised.csv', index = False)